# FAQ Topic Generation

Topic generation using Latent Dirichlet Allocation (LDA)

Author: Shreyash Gupta

Organization: IndiaMART InterMESH Pvt. Ltd.

# Importing Dataset

Importing necessary modules

In [1]:
import pandas as pd

Reading the input data

In [2]:
df = pd.read_excel("faqtrain.xlsx")

# Data preprocessing

Importing necessary modules

In [3]:
import nltk
nltk.download("stopwords")
import re
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creating a list of all question titles

In [4]:
data = df["Question Title"].values.tolist()

### Defining clean-up functions

Removing e-mails

In [5]:
def remove_emails(text):
    text = [re.sub("\S*@\S*\s?", "", sent) for sent in text]
    return text

Removing extra spaces

In [6]:
def remove_extra_spaces(text):
    text = [re.sub("\s+", " ", sent) for sent in text]
    return text

Removing quotes

In [7]:
def remove_quotes(text):
    text = [re.sub("\'", "", sent) for sent in text]
    return text

Tokenizing, lowercasing and removing punctuations

In [8]:
def tokenize_and_clean(text):
    for sent in text:
        yield(gensim.utils.simple_preprocess(str(sent),
                                             deacc=True))

Stopwords removal

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text]

Creating bigrams

In [10]:
def make_bigrams(text):
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in text]

Lemmatizing text

In [11]:
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in text:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Calling all functions

In [12]:
data = remove_emails(data)
data = remove_extra_spaces(data)
data = remove_quotes(data)
data_words = list(tokenize_and_clean(data))
data_words = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Topic modelling

Creating dictionary

In [13]:
id2word = corpora.Dictionary(data_lemmatized)

Creating corpus

In [14]:
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

Building the model

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 20, 
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = 'auto',
                                           per_word_topics = True)

# Model analysis

Importing necessary modules

In [16]:
from gensim.models import CoherenceModel

Printing topics

In [17]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.245*"air" + 0.087*"need" + 0.071*"coin" + 0.052*"cool" + 0.046*"flow" + '
  '0.039*"buy" + 0.036*"dry" + 0.032*"business" + 0.016*"year" + '
  '0.009*"start"'),
 (1,
  '0.073*"buyer" + 0.040*"wheat" + 0.039*"center" + 0.034*"purchase" + '
  '0.000*"cashew_nuts" + 0.000*"seed" + 0.000*"bacoco" + 0.000*"power" + '
  '0.000*"cord" + 0.000*"grass"'),
 (2,
  '0.106*"sell" + 0.095*"test" + 0.060*"online" + 0.000*"old" + '
  '0.000*"apparatus" + 0.000*"go" + 0.000*"mechanic" + 0.000*"could" + '
  '0.000*"nebula" + 0.000*"payment"'),
 (3,
  '0.142*"light" + 0.086*"lead" + 0.010*"white" + 0.000*"pair" + 0.000*"metal" '
  '+ 0.000*"scrap" + 0.000*"tube_light" + 0.000*"watt" + 0.000*"set" + '
  '0.000*"car"'),
 (4,
  '0.000*"axel" + 0.000*"changeover_switch" + 0.000*"salfee" + 0.000*"bokaro" '
  '+ 0.000*"alambagh" + 0.000*"micropore" + 0.000*"residence" + 0.000*"cft" + '
  '0.000*"contract" + 0.000*"vl"'),
 (5,
  '0.071*"home" + 0.059*"supplier" + 0.051*"list" + 0.032*"silk" + '
  '0.0

Compute perplexity

In [18]:
print('Perplexity: ', lda_model.log_perplexity(corpus))

Perplexity:  -19.349753593068733


Compute coherence score

In [19]:
coherence_model_lda = CoherenceModel(model = lda_model,
                                     texts = data_lemmatized,
                                     dictionary = id2word,
                                     coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.49526484124977904


# Model visualization

Importing necessary modules

In [20]:
import pyLDAvis
import pyLDAvis.gensim

Visualizng topic - keywords relationship

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Pooja\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.592947  0.129678       1        1  81.027725
19     0.168305 -0.392564       2        1   5.561580
0      0.020593  0.013395       3        1   3.116227
8     -0.043596  0.016787       4        1   1.560644
13    -0.020975  0.018351       5        1   1.299786
16     0.003667  0.016370       6        1   1.106418
11    -0.053497  0.013877       7        1   0.863158
14    -0.043414  0.016770       8        1   0.756749
5     -0.051157  0.014835       9        1   0.685167
7     -0.045315  0.016442      10        1   0.659546
10    -0.042564  0.016947      11        1   0.523368
18    -0.053594  0.013809      12        1   0.470650
3     -0.052054  0.014461      13        1   0.435722
2     -0.050513  0.015020      14        1   0.415345
6     -0.049122  0.015443      15        1   0.334543
12    -0.055986  0.012584      16        1   0.307664
1     -0.054953  0.013179      17        1   0.287690
15    -0.049805  0.015231      18        1   0.258139
9     -0.057818  0.011363      19        1   0.206309
4     -0.061148  0.008022      20        1   0.123567, topic_info=     Category          Freq           Term         Total  loglift  logprob
36    Default   3519.000000        machine   3519.000000  30.0000  30.0000
5     Default  26197.000000          price  26197.000000  29.0000  29.0000
277   Default  25679.000000           good  25679.000000  28.0000  28.0000
595   Default  19186.000000          piece  19186.000000  27.0000  27.0000
162   Default    683.000000            air    683.000000  26.0000  26.0000
41    Default    343.000000         powder    343.000000  25.0000  25.0000
105   Default    288.000000           want    288.000000  24.0000  24.0000
161   Default    281.000000            bag    281.000000  23.0000  23.0000
206   Default    268.000000             kg    268.000000  22.0000  22.0000
147   Default    234.000000           pump    234.000000  21.0000  21.0000
158   Default    243.000000           need    243.000000  20.0000  20.0000
37    Default    293.000000           make    293.000000  19.0000  19.0000
133   Default    141.000000            bed    141.000000  18.0000  18.0000
2     Default    197.000000           coin    197.000000  17.0000  17.0000
148   Default    136.000000            get    136.000000  16.0000  16.0000
165   Default    145.000000           cool    145.000000  15.0000  15.0000
166   Default    128.000000           flow    128.000000  14.0000  14.0000
181   Default     80.000000      generator     80.000000  13.0000  13.0000
128   Default     86.000000          mixer     86.000000  12.0000  12.0000
29    Default     68.000000           rate     68.000000  11.0000  11.0000
138   Default    109.000000            buy    109.000000  10.0000  10.0000
150   Default     61.000000          chair     61.000000   9.0000   9.0000
159   Default     58.000000        product     58.000000   8.0000   8.0000
97    Default     68.000000           unit     68.000000   7.0000   7.0000
139   Default    100.000000            dry    100.000000   6.0000   6.0000
193   Default     71.000000           work     71.000000   5.0000   5.0000
177   Default     56.000000          light     56.000000   4.0000   4.0000
137   Default     89.000000       business     89.000000   3.0000   3.0000
207   Default     65.000000           much     65.000000   2.0000   2.0000
83    Default     70.000000           type     70.000000   1.0000   1.0000
...       ...           ...            ...           ...      ...      ...
3357  Topic20      0.011367      pocketing      0.886439   2.3397  -9.1819
3245  Topic20      0.011367            pav      0.886439   2.3397  -9.1819
3243  Topic20      0.011367           skim      0.886439   2.3397  -9.1819
3242  Topic20      0.011367         turkey      0.886439   2.3397  -9.1819
3241  Topic20      0.011367      dehydrate      0.886439   2.3397  -9